In [1]:
import numpy as np
import pandas as pd
from os import path
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
import sklearn
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn import preprocessing, metrics
from sklearn.metrics import (roc_curve, auc, accuracy_score, precision_score, 
                             recall_score, f1_score, balanced_accuracy_score, 
                             matthews_corrcoef)
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
import shap
from imblearn.over_sampling import RandomOverSampler
import innvestigate

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

tf.compat.v1.disable_eager_execution()

2025-02-21 07:04:59.167025: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 07:04:59.467939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-21 07:05:00.262819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
def confusion_metrics (name_model,predictions,true_labels):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

    return Acc, Precision, Recall, F1, BACC, MCC

In [3]:
output_file_name = "CIC_Base.txt"


In [4]:

req_cols = [
    
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',
    ' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate'
    , ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', ' CWE Flag Count', 'Fwd IAT Total'
    
    , ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean', 'Fwd PSH Flags', ' Fwd IAT Min'
     
    ,  ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean', 'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets', ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max'
            
            , ' Label']

# Information gain top 10 features
top10 = [' Average Packet Size',
          ' Packet Length Std', 
          ' Packet Length Variance', 
          ' Packet Length Mean',
            ' Destination Port', 
            ' Subflow Bwd Bytes', 
            ' Total Length of Bwd Packets', 
            ' Avg Bwd Segment Size', 
            ' Bwd Packet Length Mean',  
            'Bwd Packet Length Max', 
            ' Label']

# top_40_features = [
#     "Average Packet Size", "Packet Length Variance", "Packet Length Std", "Packet Length Mean",
#     "Destination Port", "Total Length of Bwd Packets", "Subflow Bwd Bytes", "Avg Bwd Segment Size",
#     "Bwd Packet Length Mean", "Bwd Packet Length Max", "Subflow Fwd Bytes", "Total Length of Fwd Packets",
#     "Init_Win_bytes_backward", "Init_Win_bytes_forward", "Max Packet Length", "Fwd Packet Length Max",
#     "Flow IAT Max", "Bwd Packets/s", "Flow Duration", "Fwd IAT Max",
#     "Bwd Header Length", "Fwd Packet Length Mean", "Avg Fwd Segment Size", "Fwd Packets/s",
#     "Fwd Header Length", "Fwd IAT Total", "Flow IAT Mean", "Fwd IAT Mean",
#     "Bwd Packet Length Std", "Total Backward Packets", "Subflow Bwd Packets", "Fwd Packet Length Std",
#     "Flow IAT Std", "Bwd Packet Length Min", "Bwd IAT Max", "Fwd IAT Std",
#     "Bwd IAT Total", "Total Fwd Packets", "Subflow Fwd Packets", "Bwd IAT Mean"
# ]

top_40_features = [
    ' Average Packet Size', 
    ' Packet Length Variance', 
    ' Packet Length Std', 
    ' Packet Length Mean',
    ' Destination Port', 

    ' Total Length of Bwd Packets', 
    ' Subflow Bwd Bytes', 
    ' Avg Bwd Segment Size',
    ' Bwd Packet Length Mean', 
    ' Bwd Packet Length Max', 
    
    ' Subflow Fwd Bytes', 
    ' Total Length of Fwd Packets',
    ' Init_Win_bytes_backward', 
    ' Init_Win_bytes_forward', 
    ' Max Packet Length', 
    
    ' Fwd Packet Length Max',
    ' Flow IAT Max', 
    ' Bwd Packets/s', 
    ' Flow Duration', 
    ' Fwd IAT Max',
    ' Bwd Header Length', ' Fwd Packet Length Mean', ' Avg Fwd Segment Size', ' Fwd Packets/s',
    ' Fwd Header Length', ' Fwd IAT Total', ' Flow IAT Mean', ' Fwd IAT Mean',
    ' Bwd Packet Length Std', ' Total Backward Packets', ' Subflow Bwd Packets', ' Fwd Packet Length Std',
    ' Flow IAT Std', ' Bwd Packet Length Min', ' Bwd IAT Max', ' Fwd IAT Std',
    ' Bwd IAT Total', ' Total Fwd Packets', ' Subflow Fwd Packets', ' Bwd IAT Mean', ' Label'
]



# usecols=top_40_features
# req_cols = top10

# req_cols = [' Destination Port',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Label']
#---------------------------------------------------------------------
#Load Databases from csv file

path_str = '/home/oarreche@ads.iu.edu/HITL/cicids/cicids_db/'
fraction = 1
#---------------------------------------------------------------------
#Load Databases from csv file
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')


df0 = pd.read_csv (path_str + 'Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv (path_str + 'Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv (path_str +'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv (path_str +'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv (path_str +'Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv (path_str +'Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5, df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac = 0.333)



#---------------------------------------------------------------------
# Normalize database
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled = df.copy()

y = df_max_scaled[' Label'].replace({'DDoS' :'Dos/Ddos' ,'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop(' Label')


df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

#---------------------------------------------------------------------





---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------



In [5]:
# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)

df = X.assign( Label = y)

y, label = pd.factorize(y)

# Separate Training and Testing db

---------------------------------------------------------------------------------
Separating features and labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 756652, 'Dos/Ddos': 127047, 'PortScan': 52987, 'Brute Force': 4594, 'Web Attack': 716, 'Bot': 628, 'Infiltration': 13})


In [6]:
from sklearn.feature_selection import mutual_info_classif

# Assume X is your feature matrix and y is the target variable
info_gain = mutual_info_classif(X, y)

# Convert to a Pandas DataFrame for better visualization
info_gain_df = pd.DataFrame({'Feature': X.columns, 'Information Gain': info_gain})
info_gain_df = info_gain_df.sort_values(by='Information Gain', ascending=False)

# Display top features based on Information Gain
# print(info_gain_df)
print(info_gain_df.to_string())  # Ensures no truncation


                         Feature  Information Gain
52           Average Packet Size          0.478872
42        Packet Length Variance          0.476843
41             Packet Length Std          0.476447
40            Packet Length Mean          0.450019
0               Destination Port          0.431400
5    Total Length of Bwd Packets          0.423641
64             Subflow Bwd Bytes          0.423626
54          Avg Bwd Segment Size          0.413709
12        Bwd Packet Length Mean          0.413208
10         Bwd Packet Length Max          0.387337
62             Subflow Fwd Bytes          0.384109
4    Total Length of Fwd Packets          0.383956
66       Init_Win_bytes_backward          0.383346
65        Init_Win_bytes_forward          0.381655
39             Max Packet Length          0.355453
6          Fwd Packet Length Max          0.343933
18                  Flow IAT Max          0.332244
37                 Bwd Packets/s          0.323523
1                  Flow Duratio

In [7]:
print(info_gain_df.to_string())  # Ensures no truncation
# 

                         Feature  Information Gain
52           Average Packet Size          0.478872
42        Packet Length Variance          0.476843
41             Packet Length Std          0.476447
40            Packet Length Mean          0.450019
0               Destination Port          0.431400
5    Total Length of Bwd Packets          0.423641
64             Subflow Bwd Bytes          0.423626
54          Avg Bwd Segment Size          0.413709
12        Bwd Packet Length Mean          0.413208
10         Bwd Packet Length Max          0.387337
62             Subflow Fwd Bytes          0.384109
4    Total Length of Fwd Packets          0.383956
66       Init_Win_bytes_backward          0.383346
65        Init_Win_bytes_forward          0.381655
39             Max Packet Length          0.355453
6          Fwd Packet Length Max          0.343933
18                  Flow IAT Max          0.332244
37                 Bwd Packets/s          0.323523
1                  Flow Duratio

In [ ]:
#                         Feature  Information Gain
# 52           Average Packet Size          0.478872
# 42        Packet Length Variance          0.476843
# 41             Packet Length Std          0.476447
# 40            Packet Length Mean          0.450019
# 0               Destination Port          0.431400
# 5    Total Length of Bwd Packets          0.423641
# 64             Subflow Bwd Bytes          0.423626
# 54          Avg Bwd Segment Size          0.413709
# 12        Bwd Packet Length Mean          0.413208
# 10         Bwd Packet Length Max          0.387337
# 62             Subflow Fwd Bytes          0.384109
# 4    Total Length of Fwd Packets          0.383956
# 66       Init_Win_bytes_backward          0.383346
# 65        Init_Win_bytes_forward          0.381655
# 39             Max Packet Length          0.355453
# 6          Fwd Packet Length Max          0.343933
# 18                  Flow IAT Max          0.332244
# 37                 Bwd Packets/s          0.323523
# 1                  Flow Duration          0.317895
# 23                   Fwd IAT Max          0.311460
# 35             Bwd Header Length          0.309675
# 8         Fwd Packet Length Mean          0.304639
# 53          Avg Fwd Segment Size          0.304341
# 36                 Fwd Packets/s          0.301550
# 34             Fwd Header Length          0.299453
# 20                 Fwd IAT Total          0.297142
# 16                 Flow IAT Mean          0.294122
# 21                  Fwd IAT Mean          0.288025
# 13         Bwd Packet Length Std          0.250428
# 3         Total Backward Packets          0.245154
# 63           Subflow Bwd Packets          0.245056
# 9          Fwd Packet Length Std          0.236534
# 17                  Flow IAT Std          0.230764
# 11         Bwd Packet Length Min          0.224931
# 28                   Bwd IAT Max          0.218356
# 22                   Fwd IAT Std          0.211027
# 25                 Bwd IAT Total          0.203711
# 2              Total Fwd Packets          0.202003
# 61           Subflow Fwd Packets          0.201302
# 26                  Bwd IAT Mean          0.191731
# 7          Fwd Packet Length Min          0.162821
# 38             Min Packet Length          0.161690
# 24                   Fwd IAT Min          0.160989
# 69                   Active Mean          0.160002
# 72                    Active Min          0.159630
# 71                    Active Max          0.158500
# 29                   Bwd IAT Min          0.157398
# 68          min_seg_size_forward          0.156843
# 19                  Flow IAT Min          0.149903
# 75                      Idle Max          0.145323
# 27                   Bwd IAT Std          0.140486
# 51                 Down/Up Ratio          0.140476
# 73                     Idle Mean          0.140197
# 76                      Idle Min          0.139595
# 67              act_data_pkt_fwd          0.128589
# 46                PSH Flag Count          0.102827
# 47                ACK Flag Count          0.086046
# 74                      Idle Std          0.026023
# 48                URG Flag Count          0.025466
# 43                FIN Flag Count          0.023065
# 70                    Active Std          0.017445
# 30                 Fwd PSH Flags          0.010793
# 44                SYN Flag Count          0.010748
# 14                  Flow Bytes/s          0.000471
# 56          Fwd Avg Packets/Bulk          0.000217
# 32                 Fwd URG Flags          0.000161
# 57             Fwd Avg Bulk Rate          0.000067
# 50                ECE Flag Count          0.000057
# 33                 Bwd URG Flags          0.000000
# 15                Flow Packets/s          0.000000
# 60             Bwd Avg Bulk Rate          0.000000
# 58            Bwd Avg Bytes/Bulk          0.000000
# 55            Fwd Avg Bytes/Bulk          0.000000
# 31                 Bwd PSH Flags          0.000000
# 49                CWE Flag Count          0.000000
# 45                RST Flag Count          0.000000
# 59          Bwd Avg Packets/Bulk          0.000000

In [8]:
# top_40_features = [
#     "Average Packet Size", "Packet Length Variance", "Packet Length Std", "Packet Length Mean",
#     "Destination Port", "Total Length of Bwd Packets", "Subflow Bwd Bytes", "Avg Bwd Segment Size",
#     "Bwd Packet Length Mean", "Bwd Packet Length Max", "Subflow Fwd Bytes", "Total Length of Fwd Packets",
#     "Init_Win_bytes_backward", "Init_Win_bytes_forward", "Max Packet Length", "Fwd Packet Length Max",
#     "Flow IAT Max", "Bwd Packets/s", "Flow Duration", "Fwd IAT Max",
#     "Bwd Header Length", "Fwd Packet Length Mean", "Avg Fwd Segment Size", "Fwd Packets/s",
#     "Fwd Header Length", "Fwd IAT Total", "Flow IAT Mean", "Fwd IAT Mean",
#     "Bwd Packet Length Std", "Total Backward Packets", "Subflow Bwd Packets", "Fwd Packet Length Std",
#     "Flow IAT Std", "Bwd Packet Length Min", "Bwd IAT Max", "Fwd IAT Std",
#     "Bwd IAT Total", "Total Fwd Packets", "Subflow Fwd Packets", "Bwd IAT Mean"
# ]
# remaining_features = [
#     "Fwd Packet Length Min", "Min Packet Length", "Fwd IAT Min", "Active Mean",
#     "Active Min", "Active Max", "Bwd IAT Min", "min_seg_size_forward",
#     "Flow IAT Min", "Idle Max", "Bwd IAT Std", "Down/Up Ratio",
#     "Idle Mean", "Idle Min", "act_data_pkt_fwd", "PSH Flag Count",
#     "ACK Flag Count", "Idle Std", "URG Flag Count", "FIN Flag Count",
#     "Active Std", "Fwd PSH Flags", "SYN Flag Count", "Flow Bytes/s",
#     "Fwd Avg Packets/Bulk", "Fwd URG Flags", "Fwd Avg Bulk Rate", "ECE Flag Count",
#     "Bwd URG Flags", "Flow Packets/s", "Bwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk",
#     "Fwd Avg Bytes/Bulk", "Bwd PSH Flags", "CWE Flag Count", "RST Flag Count",
#     "Bwd Avg Packets/Bulk"
# ]
